In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [21]:
def create_link(ecli_from, ecli_to, sparql):
    base_query = '''
    prefix dcterm: <http://purl.org/dc/terms/>
    insert data 
    {{<http://deeplink.rechtspraak.nl/uitspraak?id={}>
        dcterm:references 
        <http://deeplink.rechtspraak.nl/uitspraak?id={}> 
    .}}'''
    query = base_query.format(ecli_from, ecli_to)
    sparql.method = 'POST'
    sparql.setQuery(query)
    res = sparql.query()

In [31]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [32]:
#Create for Gijs' network

import pandas as pd
filepath_csv = '/media/sf_VBox_Shared/CaseLaw/graphs/test3 wg aanspr.csv'
links_wgaans = pd.read_csv(filepath_csv, sep=';', header=0)
links_wgaans.columns = ['id', 'reference']
links_wgaans = links_wgaans.drop_duplicates()

In [33]:
for i, r in links_wgaans.iterrows():
    create_link(r['id'], r['reference'], sparql)

In [35]:
#Make a query
queryString = "select (COUNT(*) as ?count) {?s ?p ?o}"
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
ret = sparql.query()
ret.convert()

{'head': {'vars': ['count']},
 'results': {'bindings': [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '4037389'}}]}}